In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Two sets of data are analyzed in this notebook.
# 1. This is a time series data of COVID-19 status in terms of the age in South Korea


## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import re
import datetime
import folium
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas import GeoDataFrame
import math
from folium.plugins import HeatMap, MarkerCluster
from folium import Marker,GeoJson,Choropleth, Circle

## Import data and view first few rows

In [ ]:
time_age = pd.read_csv('/kaggle/input/coronavirusdataset/TimeAge.csv')
time_gender = pd.read_csv('/kaggle/input/coronavirusdataset/TimeGender.csv')


In [ ]:
time_age.head()

## Removing the string from the 'age' column to get only integers

In [ ]:
time_age['age'] =  time_age['age'].str.rstrip('s')

In [ ]:
time_age.head()

In [ ]:
# Renaming the 'age' column

time_age.rename(columns = {'age':'age_group'}, inplace = True)

In [ ]:
time_age.head()

## A plot of age group and confirmed cases show those in their 20s with the highest numbers

In [ ]:
sns.boxplot('age_group', 'confirmed', data = time_age) 


## A line plot of the age group and deceased show the number of deaths increase with age

In [ ]:
sns.lineplot('age_group', 'deceased', data = time_age)  

## More people with confirmed Covid-19 diagnosis in their 80s or 90s died. And then those in the 60s died, then the 30s and so on
### So the younger you are the less likely you'll die if you're confirmed.

In [ ]:
sns.scatterplot('confirmed', 'deceased', data = time_age, hue = 'age_group')


## The distribution of date in the dataset is below. 
### Each date occurs more than once.


In [ ]:
time_age.date.value_counts()

## So I'll group by the date and take a look at what the data shows 

In [ ]:
time = time_age.groupby('date', as_index = True).sum()

In [ ]:
time 

## It showed that as the days progress the number of confirmed cases increased and so did the deceased

In [ ]:
sns.lineplot('confirmed', 'deceased', data = time) # The number of deaths was definitely reaching a plateau

## Checking the distribution of time. It's only 0. So this is an observation of data within a time frame on different days

In [ ]:
time_age.time.value_counts() 

# 2. This data is a time series of COVID-19 status in terms of gender in South Korea

## So within a given time frame, this anaylis was also done
### View first few rows of the data

In [ ]:
time_gender.head() 

In [ ]:
time_gender.time.value_counts()

## A plot shows there are more females with confirmed cases than males

In [ ]:
sns.violinplot('sex', 'confirmed', data = time_gender)  

## But more males died than females

In [ ]:
sns.violinplot('sex', 'deceased', data = time_gender)  

### The graph below confirms the above 2 graphs

In [ ]:
sns.scatterplot('deceased', 'confirmed', data = time_gender, hue = 'sex')
 

## A graph of date and deceased show a climb in deaths as time progresses

In [ ]:
plt.figure(figsize = (20, 18))
sns.scatterplot('deceased', 'date', data = time_gender, hue = 'sex')
 

## Grouping by the date and summing it up gives the total deceased

In [ ]:
gender = time_gender.groupby('date').sum()

In [ ]:
gender

## The graph below shows the total diseased versus confirmed

In [ ]:
sns.scatterplot('deceased', 'confirmed', data = gender)
 